# Python code to create a quantized index

In [1]:
from autofaiss.external.quantize import Quantizer
from autofaiss.external.optimize import get_optimal_index_keys
quantizer = Quantizer()

## 1. (Optional) Use heuristic to find an index that matchs your memory requirements

The indices are sorted by decreasing size.  
If you let the heuristic choose for you in the quantize function (next step), it will just choose the first one.

More info about how to choose an index here : https://github.com/facebookresearch/faiss/wiki/Guidelines-to-choose-an-index

In [2]:
print(get_optimal_index_keys(nb_vectors=200_000_000,
                             dim_vector=1280,
                             max_index_memory_usage="20G")[:3])

['OPQ64_256,IVF131072_HNSW32,PQ64x8', 'OPQ64_256,IVF65536_HNSW32,PQ64x8', 'OPQ64_256,IVF262144_HNSW32,PQ64x8']


## 2. Create the index that maximizes the recalls given speed and memory constraints

The index created will be saved on the hdfs and on local disk at `local_embeddings_path` path.

You can try it following the instructions in the `use_index.ipynb` notebook.

In [3]:
# Fix a specific index or let the heuristic choose for you given the memory constaint
index_key = ["OPQ32_128,IVF131072_HNSW32,PQ32x8", None][1]

# Fix specific search params or let the heuristic choose for you given the speed constaint
index_param = ["nprobe=16,efSearch=32,ht=2048", None][1]

# Launch the quantization pipeline
quantizer.quantize(embeddings_path="/user/deepr/dev/r.beaumont/image_embeddings_100_views_US",
                   output_path="/user/v.paltz/tests_autofaiss/image_embeddings_100_views_US",
                   local_embeddings_path="/home/v.paltz/downloaded_vectors/image_embeddings_100_views_US",
                   index_key=index_key,
                   index_param=index_param,
                   max_index_query_time_ms=5.0,
                   max_index_memory_usage="18G",
                   current_memory_available="300G",
                   use_gpu=False,
                   metric_type="ip")

Checking that your have enough memory available to create the index
>>> Finished in 3.8933 secs
Download the parquet files on local disk


100%|██████████| 926/926 [00:10<00:00, 84.40it/s]

>>> Finished in 11.0367 secs
Convert .parquet files to numpy arrays



100%|██████████| 926/926 [01:48<00:00,  8.52it/s]

>>> Finished in 108.7071 secs
Computing inverse mapping between indice and (productID, partnerID)


...

100%|██████████| 926/926 [06:56<00:00,  2.22it/s]


	>>> Finished in 417.2501 secs
>>> Finished in 1497.8998 secs
Computing best hyperparameters
>>> Finished in 35.3311 secs
The best hyperparameters are: nprobe=128,efSearch=256,ht=2048
Saving index on hdfs
>>> Finished in 20.4893 secs
Saving the index on local disk
>>> Finished in 1.3484 secs
Compute fast metrics
>>> Finished in 13.4495 secs
Intermediate recap:
{'average speed': '4.16 ms/query',
 'compression ratio': 66,
 'nb vectors': 32431891,
 'reconstruction error': '45.2587%',
 'size': '2.3G',
 'vectors dimension': 1280}
Compute medium metrics
	-> Compute small ground truth


100%|██████████| 926/926 [06:38<00:00,  2.33it/s]


	>>> Finished in 411.0809 secs
	-> Compute recalls
	>>> Finished in 2.4312 secs
>>> Finished in 413.6411 secs

Index OPQ64_256,IVF131072_HNSW32,PQ64x8 built, performances recap:
{'1-recall@20': 0.868,
 '1-recall@40': 0.92,
 '20-recall@20': 0.46840000000000004,
 '40-recall@40': 0.48815,
 'average speed': '4.16 ms/query',
 'compression ratio': 66,
 'nb vectors': 32431891,
 'reconstruction error': '45.2587%',
 'size': '2.3G',
 'vectors dimension': 1280}


'Done'